In [3]:
import sys
!{sys.executable} -m pip install sqlite-utils

In [10]:
!wget http://db.csail.mit.edu/labdata/data.txt.gz
!gunzip data.txt.gz
!sed -i '1s/^/day time_of_day epoch moteid temperature humidity light voltage\n/' data.txt
!head data.txt

--2021-11-21 15:52:03--  http://db.csail.mit.edu/labdata/data.txt.gz
Resolving db.csail.mit.edu (db.csail.mit.edu)... 128.52.128.91
Connecting to db.csail.mit.edu (db.csail.mit.edu)|128.52.128.91|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34422518 (33M) [application/x-gzip]
Saving to: ‘data.txt.gz’

data.txt.gz         100%[===================>]  32.83M   158MB/s    in 0.2s    

2021-11-21 15:52:03 (158 MB/s) - ‘data.txt.gz’ saved [34422518/34422518]

day time_of_day epoch moteid temperature humidity light voltage
2004-03-31 03:38:15.757551 2 1 122.153 -3.91901 11.04 2.03397
2004-02-28 00:59:16.02785 3 1 19.9884 37.0933 45.08 2.69964
2004-02-28 01:03:16.33393 11 1 19.3024 38.4629 45.08 2.68742
2004-02-28 01:06:16.013453 17 1 19.1652 38.8039 45.08 2.68742
2004-02-28 01:06:46.778088 18 1 19.175 38.8379 45.08 2.69964
2004-02-28 01:08:45.992524 22 1 19.1456 38.9401 45.08 2.68742
2004-02-28 01:09:22.323858 23 1 19.1652 38.872 45.08 2.68742
2004-02-28 01:09:46.1

In [11]:
!sqlite-utils insert intel-sensor.sqlite readings data.txt --csv --sniff --detect-types

  [####################################]  100%          

In [18]:
!sqlite-utils schema intel-sensor.sqlite

CREATE TABLE "readings" (
   [day] TEXT,
   [time_of_day] TEXT,
   [epoch] INTEGER,
   [moteid] INTEGER,
   [temperature] FLOAT,
   [humidity] FLOAT,
   [light] FLOAT,
   [voltage] FLOAT
);


In [20]:
from sqlalchemy import create_engine
from datools.explanation.algorithms import diff
from datools.models import Column
from datools.sqlalchemy_utils import query_results_pretty_print

engine = create_engine('sqlite:///intel-sensor.sqlite')

## Including range-balued attributes doesn't replicate the Scorpion paper
The current implementation only handles equality predicates, so range-valued attributes like humidity, light, voltage get an "advantage" because specific values of those measurements likely also highly correlate with failing motes. Still, moteid=15 (the one identified in the paper) is the 4th top explanation, behind voltage/humidity/voltage.

In [23]:
candidates = diff(
        engine=engine,
        test_relation='SELECT moteid, temperature, humidity, light, voltage FROM readings WHERE temperature > 100 AND day > "2004-03-01" and day < "2004-03-10"',
        control_relation='SELECT moteid, temperature, humidity, light, voltage FROM readings WHERE temperature <= 100 AND day > "2004-03-01" and day < "2004-03-10"',
        on_columns=(Column('moteid'), Column('humidity'), Column('light'), Column('voltage'),),
        min_support=0.05,
        min_risk_ratio=2.0,
        max_order=1)
for candidate in candidates:
    print(candidate)

Explanation(predicates=(Predicate(left=Column(name='voltage'), operator=<Operator.EQUALS: 1>, right=2.32),), risk_ratio=732.5350873788292)
Explanation(predicates=(Predicate(left=Column(name='humidity'), operator=<Operator.EQUALS: 1>, right=-3.91901),), risk_ratio=581.896923076923)
Explanation(predicates=(Predicate(left=Column(name='voltage'), operator=<Operator.EQUALS: 1>, right=2.31097),), risk_ratio=413.2085152838428)
Explanation(predicates=(Predicate(left=Column(name='moteid'), operator=<Operator.EQUALS: 1>, right=15),), risk_ratio=404.8320855614973)
Explanation(predicates=(Predicate(left=Column(name='humidity'), operator=<Operator.EQUALS: 1>, right=-4.0),), risk_ratio=375.40059494298464)
Explanation(predicates=(Predicate(left=Column(name='voltage'), operator=<Operator.EQUALS: 1>, right=2.30202),), risk_ratio=361.1173104434907)
Explanation(predicates=(Predicate(left=Column(name='voltage'), operator=<Operator.EQUALS: 1>, right=2.3291),), risk_ratio=269.18333022781434)
Explanation(pre

# When only considering moteids, we replicate the Scorpion paper
If we only consider moteid (the only set-valued attribute), the Scorpion result replicates: moteid=15 is the offender!

In [22]:
candidates = diff(
        engine=engine,
        test_relation='SELECT moteid, temperature, humidity, light, voltage FROM readings WHERE temperature > 100 AND day > "2004-03-01" and day < "2004-03-10"',
        control_relation='SELECT moteid, temperature, humidity, light, voltage FROM readings WHERE temperature <= 100 AND day > "2004-03-01" and day < "2004-03-10"',
        on_columns=(Column('moteid'),),
        min_support=0.05,
        min_risk_ratio=2.0,
        max_order=1)
for candidate in candidates:
    print(candidate)

Explanation(predicates=(Predicate(left=Column(name='moteid'), operator=<Operator.EQUALS: 1>, right=15),), risk_ratio=404.8320855614973)
Explanation(predicates=(Predicate(left=Column(name='moteid'), operator=<Operator.EQUALS: 1>, right=18),), risk_ratio=200.5765335449176)


In [9]:
!rm data.txt* intel-sensor.sqlite*

rm: cannot remove 'data.txt*': No such file or directory
rm: cannot remove 'intel-sensor.sqlite*': No such file or directory
